In [13]:
%pip install pingouin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.4/204.4 kB 6.2 MB/s eta 0:00:00


In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
penguins = sns.load_dataset("penguins")
penguins = penguins.dropna()

In [14]:
import pingouin as pg

anova_results = pg.anova(dv='flipper_length_mm', between='species', data=penguins, detailed=True)
print(anova_results)

    Source            SS   DF            MS           F          p-unc  \
0  species  50525.883615    2  25262.941807  567.406992  1.587418e-107   
1   Within  14692.753022  330     44.523494         NaN            NaN   

        np2  
0  0.774715  
1       NaN  


In [16]:
overall_mean = penguins['flipper_length_mm'].mean()
group_means = penguins.groupby('species')['flipper_length_mm'].mean()
group_sizes = penguins.groupby('species')['flipper_length_mm'].size()

SSB = sum((group_means - overall_mean)**2 * group_sizes)
SSW = sum(((penguins[penguins['species'] == species]['flipper_length_mm'] - group_means[species])**2).sum() for species in group_means.index)

df_between = len(group_means) - 1
df_within = len(penguins) - len(group_means)

MSB = SSB / df_between
MSW = SSW / df_within

F_statistic = MSB / MSW
p_value = 1 - stats.f.cdf(F_statistic, df_between, df_within)

print("SSB:", SSB)
print("SSW:", SSW)
print("df_between:", df_between)
print("df_within:", df_within)
print("MSB:", MSB)
print("MSW:", MSW)
print("F-statistic:", F_statistic)
print("p-value:", p_value)

# Conclusion based on p-value
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis. Significant differences exist between groups.")
else:
    print("Fail to reject the null hypothesis. No significant differences between groups.")

SSB: 50525.88361488001
SSW: 14692.753021756647
df_between: 2
df_within: 330
MSB: 25262.941807440006
MSW: 44.52349400532317
F-statistic: 567.4069920123429
p-value: 1.1102230246251565e-16
Reject the null hypothesis. Significant differences exist between groups.


In [22]:
# Perform pairwise t-tests with Bonferroni correction

pairwise_results = pg.pairwise_ttests(dv='flipper_length_mm', between='species', padjust='bonf', data=penguins)
print(pairwise_results)

  Contrast          A          B  Paired  Parametric          T         dof  \
0  species     Adelie  Chinstrap   False        True  -5.611507  120.880833   
1  species     Adelie     Gentoo   False        True -33.505504  251.350932   
2  species  Chinstrap     Gentoo   False        True -20.300894  130.589892   

  alternative         p-unc        p-corr p-adjust       BF10    hedges  
0   two-sided  1.296589e-07  3.889767e-07     bonf  1.828e+05 -0.848215  
1   two-sided  1.082642e-94  3.247927e-94     bonf  7.284e+92 -4.130274  
2   two-sided  3.208572e-42  9.625717e-42     bonf  1.838e+45 -3.141355  


/usr/local/lib/python3.10/dist-packages/pingouin/pairwise.py:28: UserWarning: pairwise_ttests is deprecated, use pairwise_tests instead.
  warnings.warn("pairwise_ttests is deprecated, use pairwise_tests instead.", UserWarning)


In [24]:
1.296589e-07*3

3.8897669999999997e-07

In [29]:
# pairwise testing

group_stats = penguins.groupby('species')['flipper_length_mm'].agg(['mean', 'var', 'size']).reset_index()
print(group_stats)

     species        mean        var  size
0     Adelie  190.102740  42.534199   146
1  Chinstrap  195.823529  50.863916    68
2     Gentoo  217.235294  43.367896   119


In [32]:
from scipy import stats
import itertools

def manual_ttest(mean1, var1, n1, mean2, var2, n2):
    # Calculate the t-statistic
    t_stat = (mean1 - mean2) / np.sqrt(var1/n1 + var2/n2)

    # Degrees of freedom (Welch-Satterthwaite equation for unequal variances)
    df = ((var1/n1 + var2/n2)**2) / (((var1/n1)**2 / (n1-1)) + ((var2/n2)**2 / (n2-1)))

    # Two-tailed p-value
    p_value = 2 * (1 - stats.t.cdf(np.abs(t_stat), df))

    return t_stat, p_value, df

results = []

# Get all unique pairs of species
pairs = list(itertools.combinations(group_stats['species'], 2))

# Perform t-tests for all pairs
for species1, species2 in pairs:
    group1 = group_stats.loc[group_stats['species'] == species1].iloc[0]
    group2 = group_stats.loc[group_stats['species'] == species2].iloc[0]

    t_stat, p_value, dof = manual_ttest(
        group1['mean'], group1['var'], group1['size'],
        group2['mean'], group2['var'], group2['size']
    )

    results.append({
        'Group 1': species1,
        'Mean 1': group1['mean'],
        'Variance 1': group1['var'],
        'Size 1': group1['size'],
        'Group 2': species2,
        'Mean 2': group2['mean'],
        'Variance 2': group2['var'],
        'Size 2': group2['size'],
        't-statistic': t_stat,
        'degrees of freedom': dof,
        'p-value': p_value,
        'bonferroni p-value': min(p_value * len(pairs), 1)
    })

# Create a DataFrame to display the results
results_df = pd.DataFrame(results)
print(results_df)

     Group 1      Mean 1  Variance 1  Size 1    Group 2      Mean 2  \
0     Adelie  190.102740   42.534199     146  Chinstrap  195.823529   
1     Adelie  190.102740   42.534199     146     Gentoo  217.235294   
2  Chinstrap  195.823529   50.863916      68     Gentoo  217.235294   

   Variance 2  Size 2  t-statistic  degrees of freedom       p-value  \
0   50.863916      68    -5.611507          120.880833  1.296589e-07   
1   43.367896     119   -33.505504          251.350932  0.000000e+00   
2   43.367896     119   -20.300894          130.589892  0.000000e+00   

   bonferroni p-value  
0        3.889767e-07  
1        0.000000e+00  
2        0.000000e+00  


All penguin species have different flipper lengths at the 5% significance level.